In [1]:
from absl import app as absl_app
from nasbench import api as nasbench_api
from silence_tensorflow import silence_tensorflow
silence_tensorflow()


NASBENCH_TFRECORD = './nasbench_only108.tfrecord'
nasbench = nasbench_api.NASBench(NASBENCH_TFRECORD)


KeyboardInterrupt: 

In [ ]:
def check(op_source, op_target, fixed_statistics):
    module_operations = fixed_statistics[1]['module_operations']
    module_adjacency = fixed_statistics[1]['module_adjacency']

    op_sources = [ x for x, y in enumerate(module_operations) if y == op_source ]
    op_targets = [ x for x, y in enumerate(module_operations) if y == op_target ]

    for op_source in op_sources:
        for op_target in op_targets:
            if module_adjacency[op_source, op_target] == 1:
                return True
    return False


In [ ]:

ops = ['input', 'output', 'conv1x1-bn-relu', 'conv3x3-bn-relu', 'maxpool3x3']
ops_name ={
    'input' : 'INPUT',
    'output' : 'OUTPUT',
    'conv1x1-bn-relu': '1X1 CONV',
    'conv3x3-bn-relu': '3X3 CONV',
    'maxpool3x3': '3X3 MAXPOOL'
}
result = {'name' : 'sharpley_value', "children" : []}
for op_source in ops:
    for op_target in ops:
        if op_source == op_target == 'input' or\
            op_source==op_target=='output'or\
            op_source == 'output' or\
            op_target == 'input':
            continue

        n1 = 0
        n2 = 0
        sum_1 = 0
        sum_2 = 0

        for data in nasbench.fixed_statistics.items():
            computed_statistics = nasbench.computed_statistics[data[0]]
            print(data, computed_statistics)
            sleep(5)
            if check(op_source, op_target, data):
                n1 += 2
                sum_1 += computed_statistics[108][0]['final_test_accuracy']
                sum_1 += computed_statistics[108][1]['final_test_accuracy']
            else:
                n2 += 2
                sum_2 += computed_statistics[108][0]['final_test_accuracy']
                sum_2 += computed_statistics[108][1]['final_test_accuracy']
        res = {
            "name" : ops_name[op_source] + "to" + ops_name[op_target],
            "source" : op_source,
            "target" : op_target,
            "value": sum_1 / n1 - sum_2 / n2,
            "include_accuracy" : sum_1 / n1,
            "exclude_accuracy" : sum_2 / n2,
            
        }
        result["children"].append(res)
        
import json
with open("./result.json", "w") as outfile:
    json.dump(result, outfile)



NameError: name 'nasbench' is not defined